#Transformer


In [0]:
!pip install --upgrade tensorflow-gpu==1.13.0rc1

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls -al

In [0]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

In [5]:
!rm -rf data_in;rm -rf data_out;ls;

adc.json  gdrive  sample_data


In [0]:
import os

if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)
    
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

In [7]:
!ls -al '/content/gdrive/My Drive/Colab Notebooks/Data/'

total 800183
-rw------- 1 root root    889842 Jun 29 03:10 ChatBotData.csv
-rw------- 1 root root   1413786 Jul  9 05:57 data_configs.json
-rw------- 1 root root 117931806 Jul  9 03:58 test.csv.zip
-rw------- 1 root root  18766496 Jul  9 03:58 test_id.npy
-rw------- 1 root root 290878832 Jul  9 03:59 test_q1.npy
-rw------- 1 root root 290878832 Jul  9 03:59 test_q2.npy
-rw------- 1 root root  22202741 Jul  9 03:58 train.csv.zip
-rw------- 1 root root   2388336 Jul  9 03:58 train_label.npy
-rw------- 1 root root  37017352 Jul  9 03:58 train_q1.npy
-rw------- 1 root root  37017352 Jul  9 03:58 train_q2.npy


In [8]:
!ls -al '/content/gdrive/My Drive/Colab Notebooks/Seq2seq/'

total 38
-rw------- 1 root root  1451 Jul 12 21:35 configs.py
-rw------- 1 root root 11523 Jul 12 21:55 data.py
-rw------- 1 root root  6820 Jul  5 22:09 model_att.py
-rw------- 1 root root  4657 Jun 29 03:42 model.py
-rw------- 1 root root  9123 Jul 12 21:35 model_tra_adv.py
drwx------ 2 root root  4096 Jul 12 21:35 __pycache__


In [9]:
!wget -P '/content/gdrive/My Drive/Colab Notebooks/Seq2seq/' https://raw.githubusercontent.com/changwookjun/learningspoons/master/Seq2seq/model_tra_adv.py

--2019-07-12 21:59:02--  https://raw.githubusercontent.com/changwookjun/learningspoons/master/Seq2seq/model_tra_adv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9123 (8.9K) [text/plain]
Saving to: ‘/content/gdrive/My Drive/Colab Notebooks/Seq2seq/model_tra_adv.py.1’

model_tra_adv.py.1  100%[===================>]   8.91K  --.-KB/s    in 0.001s  

2019-07-12 21:59:02 (9.70 MB/s) - ‘/content/gdrive/My Drive/Colab Notebooks/Seq2seq/model_tra_adv.py.1’ saved [9123/9123]



In [0]:
#!wget -P '/content/gdrive/My Drive/Colab Notebooks/Seq2seq/' https://raw.githubusercontent.com/changwookjun/learningspoons/master/Seq2seq/configs.py  

In [11]:
!wget -P '/content/gdrive/My Drive/Colab Notebooks/Seq2seq/' https://raw.githubusercontent.com/changwookjun/learningspoons/master/Seq2seq/data.py  

--2019-07-12 21:59:04--  https://raw.githubusercontent.com/changwookjun/learningspoons/master/Seq2seq/data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11523 (11K) [text/plain]
Saving to: ‘/content/gdrive/My Drive/Colab Notebooks/Seq2seq/data.py.1’

data.py.1           100%[===================>]  11.25K  --.-KB/s    in 0.001s  

2019-07-12 21:59:04 (12.2 MB/s) - ‘/content/gdrive/My Drive/Colab Notebooks/Seq2seq/data.py.1’ saved [11523/11523]



In [12]:
!cp '/content/gdrive/My Drive/Colab Notebooks/Data/ChatBotData.csv' ./data_in/ChatBotData.csv;ls -al ./data_in/

total 884
drwxr-xr-x 2 root root   4096 Jul 12 21:59 .
drwxr-xr-x 1 root root   4096 Jul 12 21:58 ..
-rw------- 1 root root 889842 Jul 12 21:59 ChatBotData.csv


In [13]:
import pandas as pd

train_data = pd.read_csv(DATA_IN_PATH + 'ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [14]:
!pip install konlpy

In [15]:
!pip install tqdm

In [0]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from konlpy.tag import Okt
import re
import os
import sys

In [0]:
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/Seq2seq/')

import data
import configs as conf
import model_tra_adv as ml

In [18]:
char2idx,  idx2char, vocabulary_length = data.load_vocabulary()

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
100%|██████████| 11823/11823 [00:11<00:00, 1062.95it/s]


In [19]:
print(vocabulary_length)

12657


In [0]:
train_input, train_label, eval_input, eval_label = data.load_data()

In [21]:
train_input_enc, train_input_enc_length = data.enc_processing(train_input, char2idx)
train_output_dec, train_output_dec_length = data.dec_input_processing(train_label, char2idx)
train_target_dec = data.dec_target_processing(train_label, char2idx)

100%|██████████| 7921/7921 [00:07<00:00, 1071.05it/s]


In [22]:
eval_input_enc, eval_input_enc_length = data.enc_processing(eval_input,char2idx)
eval_output_dec, eval_output_dec_length = data.dec_input_processing(eval_label, char2idx)
eval_target_dec = data.dec_target_processing(eval_label, char2idx)

100%|██████████| 3902/3902 [00:03<00:00, 1088.45it/s]


In [0]:
check_point_path = os.path.join(os.getcwd(), conf.DEFINES.check_point_path)
os.makedirs(check_point_path, exist_ok=True)

In [24]:
# 에스티메이터 구성한다.
transformer = tf.estimator.Estimator(
    model_fn=ml.Model,  # 모델 등록한다.
    model_dir=conf.DEFINES.check_point_path,  # 체크포인트 위치 등록한다.
    params={  # 모델 쪽으로 파라메터 전달한다.
        'embedding_size': conf.DEFINES.embedding_size,
        'model_hidden_size': conf.DEFINES.hidden_size,  
        'ffn_hidden_size': 512,
        'attention_head_size': 4,
        'learning_rate': conf.DEFINES.learning_rate,  
        'vocabulary_length': vocabulary_length,  
        'embedding_size': conf.DEFINES.embedding_size,  
        'layer_size': conf.DEFINES.layer_size,
        'max_sequence_length': conf.DEFINES.max_sequence_length,
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/check_point', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdd43b5bbe0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
transformer.train(input_fn=lambda: data.train_input_fn(train_input_enc, train_output_dec, train_target_dec, conf.DEFINES.batch_size), steps=conf.DEFINES.train_steps)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:loss = 9.376915, step = 0
INFO:tensorflow:global_step/sec: 17.569
INFO:tensorflow:loss = 1.6068469, step = 100 (5.693 sec)
INFO:tens

In [26]:
eval_result = transformer.evaluate(input_fn=lambda: data.eval_input_fn(eval_input_enc, eval_output_dec, eval_target_dec, conf.DEFINES.batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-12T22:12:39Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./data_out/check_point/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-12-22:12:42
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.85396206, global_step = 20000, loss = 2.231488
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: ./data_out/check_point/model.ckpt-20000


In [27]:
print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


EVAL set accuracy: 0.854



In [28]:
!ls -al ./data_out/check_point

total 287432
drwxr-xr-x 3 root root     4096 Jul 12 22:12 .
drwxr-xr-x 3 root root     4096 Jul 12 22:00 ..
-rw-r--r-- 1 root root      179 Jul 12 22:12 checkpoint
drwxr-xr-x 2 root root     4096 Jul 12 22:12 eval
-rw-r--r-- 1 root root 50305252 Jul 12 22:12 events.out.tfevents.1562968831.d253ede7fc88
-rw-r--r-- 1 root root 43808945 Jul 12 22:00 graph.pbtxt
-rw-r--r-- 1 root root 53885788 Jul 12 22:00 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 root root    10728 Jul 12 22:00 model.ckpt-0.index
-rw-r--r-- 1 root root 12830536 Jul 12 22:00 model.ckpt-0.meta
-rw-r--r-- 1 root root 53885788 Jul 12 22:10 model.ckpt-16760.data-00000-of-00001
-rw-r--r-- 1 root root    10728 Jul 12 22:10 model.ckpt-16760.index
-rw-r--r-- 1 root root 12830536 Jul 12 22:10 model.ckpt-16760.meta
-rw-r--r-- 1 root root 53885788 Jul 12 22:12 model.ckpt-20000.data-00000-of-00001
-rw-r--r-- 1 root root    10728 Jul 12 22:12 model.ckpt-20000.index
-rw-r--r-- 1 root root 12830536 Jul 12 22:12 model.ckpt-20000.meta


In [29]:

predic_input_enc, predic_input_enc_length = data.enc_processing(["남자친구 승진 선물로 뭐가 좋을까?"], char2idx)

predic_output_dec, predic_output_decLength = data.dec_input_processing([""], char2idx)       

predic_target_dec = data.dec_target_processing([""], char2idx)      

predictions = transformer.predict(
    input_fn=lambda:data.eval_input_fn(predic_input_enc, predic_output_dec, predic_target_dec, 1))


answer = data.pred_next_string(predictions, idx2char)
print(answer)

#answer = data.pred2string(predictions, idx2char)
#print(answer)

100%|██████████| 1/1 [00:00<00:00, 456.95it/s]

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
('평소 에 필요했던 게 좋을 것 같아요 ', True)
